In [1]:
using Pkg
Pkg.activate("libs/")
# Pkg.instantiate()
# Pkg.add("DICOM")
# Pkg.add("XLSX")
# Pkg.add("Plots")

  Activating project at `~/Desktop/Project BAC/BAC project/libs`


In [2]:
using JLD2
using DICOM
using Dates
using XLSX
using Plots
using Images

In [15]:
first_dir = "/media/molloi-lab/2TB"
BAC_root_dir = joinpath(first_dir, "BAC_Full_Data")
excel_path = "/home/molloi-lab/Desktop/Project BAC/BAC project/Ca mass calculation Final.xlsx"
V_P_dict = Dict("L CC" => 6, "L MLO" => 2, "R CC" => 14, "R MLO" => 10)

image_dirs = ["Patientstudy_BAC_2013a/data_from_KP", 
"Patientstudy_BAC_2013b/data_from_KP", 
"Patientstudy_BAC_2014/data_from_KP", 
"Patientstudy_BAC_2015a/data_from_KP", 
"Patientstudy_BAC_2015b/data_from_KP", 
"Patientstudy_BAC_2016/Data from KP"]

save_dir = joinpath(first_dir, "Clean_Dataset_full");

In [16]:
xf = XLSX.readxlsx(excel_path)
SIDs_xf = xf["Final!C:C"][2:end]
sort!(SIDs_xf)
L_CC_mass_xf = xf["Final!O:O"][2:end]
L_MLO_mass_xf = xf["Final!S:S"][2:end]
R_CC_mass_xf = xf["Final!F:F"][2:end]
R_MLO_mass_xf = xf["Final!J:J"][2:end]
xf = nothing

output csv format:

1: SID

2: path to L MLO image  
3: path to L MLO roi  
4: num_patch_with_BAC  
5: num_patch_without_BAC

6: path to L CC image  
7: path to L CC roi  
8: num_patch_with_BAC  
9: num_patch_without_BAC

10: path to R MLO image  
11: path to R MLO roi  
12: num_patch_with_BAC  
13: num_patch_without_BAC

14: path to R CC image  
15: path to R CC roi  
16: num_patch_with_BAC  
17: num_patch_without_BAC

18: total_num_patch_with_BAC  
19: total_num_patch_without_BAC

# Helper functions

In [5]:
function deduplicate_array(arr)
    idx_to_delete = []
    for j = 1 : size(arr)[1]-1
        for k = j+1 : size(arr)[1]
            if arr[j][1] == arr[k][1]
                if arr[j][3] < arr[k][3]
                    push!(idx_to_delete, j)
                else
                    push!(idx_to_delete, k)
                end
            end
        end
    end
    idx_to_keep = []
    for j = 1 : size(arr)[1]
        if !(j in idx_to_delete)
            push!(idx_to_keep, j)
        end
    end
    return arr[idx_to_keep]
end

deduplicate_array (generic function with 1 method)

In [6]:
function deduplicate_images(View_Position, i)
    # get idx from dict
    img_idx = V_P_dict[View_Position]
    # get number of images
    num_images= size(output_csv[i][img_idx])[1]
    # get number of rois
    num_rois= size(output_csv[i][img_idx+1])[1]
    if 1 == num_images == num_rois
        return "Passed"
    end
    if 1 == num_rois
        # case 1 : 1 roi vs many imgs
        # println("1 roi vs many imgs")

        roi_name = splitext(rsplit(output_csv[i][img_idx+1][1], ['_', '-', '/']; limit = 3)[end])[1]
        # println(roi_name)
        if roi_name == "zero BAC"
            # case 1.1
            img_names = []
            for img in output_csv[i][img_idx]
                img_name = splitext(rsplit(img, '/'; limit = 2)[end])[1]
                push!(img_names, img_name)
            end
            if size(unique(img_names))[1] == 1
                output_csv[i][img_idx] = [deepcopy(output_csv[i][img_idx][1])]
                return "Case 1.1"
            end
            # case 1.2 : select the latest one
            last_edit_timestamps = []
            for img in output_csv[i][img_idx]
                push!(last_edit_timestamps, get_last_edit_time(img))
            end
            img_to_keep = deepcopy(output_csv[i][img_idx][argmax(last_edit_timestamps)])
            output_csv[i][img_idx] = [img_to_keep]
            return "Case 1.1"
        end
        # case 1.3
        for img in output_csv[i][img_idx]
            img_name = splitext(rsplit(img, '/'; limit = 2)[end])[1]
            # println("\t", img_name)
            if img_name == roi_name
                output_csv[i][img_idx] = [img]
                return "Case 1.3"
            end
        end
        # case 1.4
        println("here1")
        return "Error"
    elseif 1 == num_images
        # case 2 : 1 img vs many rois
        # println("1 img vs many rois")

        last_edit_timestamps = []
        for roi in output_csv[i][img_idx+1]
            push!(last_edit_timestamps, get_last_edit_time(roi))
        end
        roi_to_keep = deepcopy(output_csv[i][img_idx+1][argmax(last_edit_timestamps)])
        output_csv[i][img_idx+1] = [roi_to_keep]
        return "Case 2"
    else
        # println("many imgs vs many rois")
        # case 3 : many imgs vs many rois
        # get names first
        img_names = []
        for img in output_csv[i][img_idx]
            img_name = splitext(rsplit(img, '/'; limit = 2)[end])[1]
            push!(img_names, [img_name, img, get_last_edit_time(img)])
        end
        roi_names = []
        for roi in output_csv[i][img_idx+1]
            roi_name = splitext(rsplit(roi, ['_', '-', '/']; limit = 3)[end])[1]
            push!(roi_names, [roi_name, roi, get_last_edit_time(roi)])
        end
        # deduplicate
        img_names = deduplicate_array(img_names)
        roi_names = deduplicate_array(roi_names)
        # find matched pair(s)
        matched_pairs = []
        for img in img_names
            for roi in roi_names
                if img[1] == roi[1]
                    push!(matched_pairs, [img[2], roi[2]])
                end
            end
        end
        if size(matched_pairs)[1] == 1
            # case 3.1 : only one matched pair
            output_csv[i][img_idx] = [deepcopy(matched_pairs[1][1])]
            output_csv[i][img_idx+1] = [deepcopy(matched_pairs[1][2])]
            return "Case 3.1"
        end
        # case 3.4 : more than one matched pair
        output_csv[i][img_idx], output_csv[i][img_idx+1] = [], []
        for pair in matched_pairs
            push!(output_csv[i][img_idx], deepcopy(pair[1]))
            push!(output_csv[i][img_idx+1], deepcopy(pair[2]))
        end
        return "Case 3.2"
    end
    # others
    return "Error"
end

deduplicate_images (generic function with 1 method)

In [7]:
function get_last_edit_time(file_path::String)
    try
        # Get the file's stat information
        file_stat = stat(file_path)
        
        # Extract the last modification time (mtime) in seconds since the epoch
        last_edit_time_epoch = file_stat.mtime
        
        # Convert the epoch time to a more readable DateTime format
        last_edit_time = Dates.unix2datetime(last_edit_time_epoch)
        
        return last_edit_time
    catch e
        println("An error occurred: ", e)
        return nothing
    end
end

get_last_edit_time (generic function with 1 method)

In [8]:
function report_csv(i)
    # SID
    curr_data = output_csv[i]
    println(curr_data[1])
    strs = ["L MLO", "L CC", "R MLO", "R CC"]
    for j = 2:4:14
        str = strs[round(Int, (j-2)/4+1)]
        println("\tpath to $str image:")
        for p in curr_data[j]
            println("\t\t", p)
        end
        println("\tpath to $str roi:")
        for p in curr_data[1+j]
            println("\t\t", p)
        end
        println("\tnum_patch_with_BAC = $(curr_data[2+j])")
        println("\tnum_patch_without_BAC = $(curr_data[3+j])")
        println()
    end
    println("\ttotal_num_patch_with_BAC = $(curr_data[18])")
    println("\tnum_patch_wtotal_num_patch_without_BACithout_BAC = $(curr_data[19])")
end

report_csv (generic function with 1 method)

In [9]:
function rec_search(root_dir, file_ext)
	files=[]
	
	# collect curr folder info
	sub_files, sub_dirs = [], []
	for f in readdir(root_dir)
		curr_path = joinpath(root_dir, f)

		if isdir(curr_path)
			push!(sub_dirs, curr_path)
		else
			push!(sub_files, curr_path)
		end
	end
	dir_ct = size(sub_dirs)[1]
	files_found = Array{Any}(undef, dir_ct)
	
	# folders: rec calls
	Threads.@threads for i = 1 : dir_ct
		files_found[i] = rec_search(sub_dirs[i], file_ext)
	end
	
	# Append after multithreading to prevent data races
	for i = 1 : dir_ct
		append!(files, files_found[i])
	end

	# files: check if it's DICOM image
	for f in sub_files
		 
		if f[end-2:end] == file_ext
			push!(files, f)
		end
	end
	
	return files
end

rec_search (generic function with 1 method)

In [10]:
function Locate_images(BAC_dir)
	println("\nLocating \"dcm\" images...")

	result = []

	for image_dir in image_dirs
		curr_dir = joinpath(BAC_dir, image_dir)
		isdir(curr_dir) || (printstyled("\tError: Failed to locate the folder: \"$image_dir\"\n"; color = :red);continue)
		print("\tFound \"")
		printstyled(image_dir; color = :yellow)
		print("\". ")
		println("\tRecursively searching images...")
		curr_files = rec_search(curr_dir, "dcm")
		append!(result, curr_files)
	end
	
	image_ct = size(result)[1]
	println("Done. Found $image_ct images.")
	return result, image_ct
end

Locate_images (generic function with 1 method)

In [11]:
function Locate_labels(BAC_dir)
	println("\nLocating \"roi\"labels...")

	label_dirs = ["Patientstudy_BAC_2013a/ROI_manual", 
				"Patientstudy_BAC_2013b/ROI_manual", 
				"Patientstudy_BAC_2014/ROI_manual", 
				"Patientstudy_BAC_2015a/ROI_manual", 
				"Patientstudy_BAC_2015b/ROI_manual", 
				"Patientstudy_BAC_2016/ROI_manual"]
	result = []

	for label_dir in label_dirs
		curr_dir = joinpath(BAC_dir, label_dir)
		isdir(curr_dir) || (printstyled("\tError: Failed to locate the folder: \"$label_dir\"\n"; color = :red);continue)
		print("\tFound \"")
		printstyled(label_dir; color = :yellow)
		print("\". ")
		println("\tRecursively searching labels...")
		curr_files = rec_search(curr_dir, "roi")
		append!(result, curr_files)
	end
	
	label_ct = size(result)[1]
	println("Done. Found $label_ct labels.")
    return result, label_ct
end

Locate_labels (generic function with 1 method)

In [12]:
function get_SID_dict()
	# Read excel
    SIDs = SIDs_xf
	# xf = XLSX.readxlsx(excel_path)
	# SIDs = xf["Final!C:C"][2:end]
	# xf = nothing

    # Create dict
    SID_dict = Dict{String, Int}()
    ct = size(SIDs)[1]
    for i = 1 : ct
        SID_dict[SIDs[i]] = i
    end

    # create output csv data
    output_csv = Array{Any, 2}(undef, ct, 19)
    Threads.@threads for i = 1 : ct
        output_csv[i] = [SIDs[i], [], [], 0, 0, [], [], 0, 0, [], [], 0, 0, [], [], 0, 0, 0, 0]
    end

    return SID_dict, ct, output_csv
end

get_SID_dict (generic function with 1 method)

In [13]:
function search_for_specific_file(file_name)
	rec_search_for_specific_file(joinpath(BAC_root_dir, "Patientstudy_BAC_2013a", "data_from_KP"), file_name)
	rec_search_for_specific_file(joinpath(BAC_root_dir, "Patientstudy_BAC_2013b", "data_from_KP"), file_name)
	rec_search_for_specific_file(joinpath(BAC_root_dir, "Patientstudy_BAC_2014", "data_from_KP"), file_name)
	rec_search_for_specific_file(joinpath(BAC_root_dir, "Patientstudy_BAC_2015a", "data_from_KP"), file_name)
	rec_search_for_specific_file(joinpath(BAC_root_dir, "Patientstudy_BAC_2015b", "data_from_KP"), file_name)
	rec_search_for_specific_file(joinpath(BAC_root_dir, "Patientstudy_BAC_2016", "Data from KP"), file_name)
end

function rec_search_for_specific_file(root_dir, file_name)
	# collect curr folder info
	sub_files, sub_dirs = [], []
	for f in readdir(root_dir)
		curr_path = joinpath(root_dir, f)

		if isdir(curr_path)
			push!(sub_dirs, curr_path)
		else
			push!(sub_files, curr_path)
		end
	end
	dir_ct = size(sub_dirs)[1]
	files_found = Array{Any}(undef, dir_ct)
	
	# folders: rec calls
	Threads.@threads for i = 1 : dir_ct
		rec_search_for_specific_file(sub_dirs[i], file_name)
	end

	# files: check if it's target_file
	for f in sub_files
		if rsplit(f, "/"; limit = 2)[2] == file_name
			@info f
		end
	end
end

rec_search_for_specific_file (generic function with 1 method)

# 1. Locate all "dcm" images and read SID, UID, etc... 

output csv format:

1: SID

2: path to L MLO image  
3: path to L MLO roi  
4: num_patch_with_BAC  
5: num_patch_without_BAC

6: path to L CC image  
7: path to L CC roi  
8: num_patch_with_BAC  
9: num_patch_without_BAC

10: path to R MLO image  
11: path to R MLO roi  
12: num_patch_with_BAC  
13: num_patch_without_BAC

14: path to R CC image  
15: path to R CC roi  
16: num_patch_with_BAC  
17: num_patch_without_BAC

18: total_num_patch_with_BAC  
19: total_num_patch_without_BAC

In [14]:
image_paths, total_num_images = Locate_images(BAC_root_dir)
SID_dict, ct, output_csv = get_SID_dict();


Locating "dcm" images...
	Error: Failed to locate the folder: "Patientstudy_BAC_2013a/data_from_KP"
	Error: Failed to locate the folder: "Patientstudy_BAC_2013b/data_from_KP"
	Error: Failed to locate the folder: "Patientstudy_BAC_2014/data_from_KP"
	Error: Failed to locate the folder: "Patientstudy_BAC_2015a/data_from_KP"
	Error: Failed to locate the folder: "Patientstudy_BAC_2015b/data_from_KP"
	Error: Failed to locate the folder: "Patientstudy_BAC_2016/Data from KP"
Done. Found 0 images.


save data to 'output_csv'

In [15]:
# runtime ~= 80 seconds
Threads.@threads for img_idx = 1 : total_num_images
# for img_idx = 1 : total_num_images
    curr_path = image_paths[img_idx]
    # Read dicom 
    dcm_data = nothing
    try
        dcm_data = dcm_parse(curr_path)
    catch e
        println("----------------------")
        println(e)
        println(curr_path)
        println("----------------------")
        continue
    end
    # Get Data
    LorR = ""
    try
        LorR = uppercase(dcm_data[(0x0020,0x0062)])
    catch e
        println("----------------------")
        println(e)
        println(curr_path)
        println("----------------------")
    end
    V_P = ""
    try
        V_P = uppercase(dcm_data[(0x0018,0x5101)])
    catch e
        println("----------------------")
        println(e)
        println(curr_path)
        println("----------------------")
    end
    curr_key = LorR*" "*V_P
    # Check V_P
    haskey(V_P_dict, curr_key) || continue

    SID = dcm_data[(0x0010,0x0020)]

    # # Shorten path to save memory and space
    # if image_paths[img_idx][1:5] == "/home"
    #     image_paths[img_idx] = image_paths[img_idx][prefix_length:end]
    # end

    # Save to 'output_csv'
    haskey(SID_dict, SID) || continue
    output_csv[SID_dict[SID]][1] = SID
    push!(output_csv[SID_dict[SID]][V_P_dict[curr_key]], curr_path)
end

# 2. Locate "roi" lables with respect to "dcm" images.

In [16]:
label_paths, total_num_labels = Locate_labels(BAC_root_dir);


Locating "roi"labels...
	Found "Patientstudy_BAC_2013a/ROI_manual". 	Recursively searching labels...
	Found "Patientstudy_BAC_2013b/ROI_manual". 	Recursively searching labels...
	Found "Patientstudy_BAC_2014/ROI_manual". 	Recursively searching labels...


	Found "Patientstudy_BAC_2015a/ROI_manual". 	Recursively searching labels...
	Found "Patientstudy_BAC_2015b/ROI_manual". 	Recursively searching labels...
	Found "Patientstudy_BAC_2016/ROI_manual". 	Recursively searching labels...
Done. Found 4614 labels.


In [17]:
found_idx = []
notfound_idx = []
# Try to match roi with image
for i = 1 : total_num_labels
    i in found_idx && continue
    curr_roi_file_name = split(label_paths[i], "/")[end]
    splited = split(curr_roi_file_name, ['-', '_'])
    View_Position, curr_roi_file_name = uppercase(splited[1]), splited[end][1:end-4]
    curr_key = View_Position[1]*" "*View_Position[2:end]
    # Check curr_key
    haskey(V_P_dict, curr_key) || (println("\"", label_paths[i], "\"\n", curr_roi_file_name,"\n");continue)
    find_idx = V_P_dict[curr_key]
    insert_idx = V_P_dict[curr_key] + 1 # idx to roi
    found = 0
    for j = 1 : ct
        curr_paths = output_csv[j][find_idx]
        for k = 1 : size(curr_paths)[1]
            img_file_name = rsplit(curr_paths[k], "/"; limit = 2)[2][1:end-4]
            if curr_roi_file_name == img_file_name
                found = k
                break
            end
        end
        if found != 0
            # # clear other paths in curr_paths
            # img_path = deepcopy(curr_paths[found])
            # output_csv[j][find_idx] = img_path
            # save roi path to output_csv
            push!(output_csv[j][insert_idx], label_paths[i])
            break
        end
    end

    if found == 0
        # println("Not found: $curr_roi_file_name")
        # println("$(label_paths[i])")
        # println("")
        push!(notfound_idx, i)
    else
        push!(found_idx, i)
    end
end

In [18]:
println("$(size(found_idx)[1]) out of $total_num_labels labels are matched.")

4614 out of 4614 labels are matched.


In [19]:
function roi_path_to_img_dir(roi_path)
    splited = split(roi_path[52:end], "/")
    year = splited[1]
    s = length(year)
    img_dir = BAC_root_dir
    for l in image_dirs
        if l[1:s] == year
            img_dir = joinpath(img_dir, l)
            break
        end 
    end
    for i = 2:size(splited)[1]
        if splited[i][1:5] == "patie"
            img_dir = joinpath(img_dir, splited[i])
            for j = i+1 : size(splited)[1]-1
                if splited[j][1:3] == "SID"
                    img_dir_ = joinpath(img_dir, splited[j][11:end])
                    if !isdir(img_dir_)
                        img_dir_ = joinpath(img_dir, splited[j])
                    end
                    img_dir = img_dir_
                    break
                end
                if splited[j][1:3] == "999"
                    img_dir = joinpath(img_dir, splited[j])
                    break
                end
            end
            break
        end
    end
    return img_dir
end


roi_path_to_img_dir (generic function with 1 method)

In [20]:
for i in notfound_idx
    currfile = label_paths[i]
    if isfile(currfile)
        println(currfile)
        # s = rsplit(currfile,"/";limit=2)
        # print(s[1][52:end],"/")
        # printstyled(s[2]; color = :yellow)
        # println("\n\n")
        # img_dir = roi_path_to_img_dir(currfile)
        # # println("\n",img_dir)
        # # println(isdir(img_dir))
        # try 
        #     run(`nautilus -q`)
        # catch e
        # end
        # if !isdir(s[1])
        #     printstyled("NOT FOUND"; color = :yellow)
        #     println(s[1])
        #     break
        # end
        # run(`xdg-open $(s[1])`)
        # if !isdir(img_dir)
        #     printstyled("NOT FOUND"; color = :yellow)
        #     println(img_dir)
        #     break
        # end
        # run(`xdg-open $img_dir`)
        # break
    end
end

# 3. Filter and deduplicate images and labels

In [21]:
function binary_search(target)
    low = 1
    high = size(SIDs_xf)[1]
    
    while low <= high
        mid = div(low + high, 2)
        
        if SIDs_xf[mid] == target
            return mid
        elseif SIDs_xf[mid] < target
            low = mid + 1
        else
            high = mid - 1
        end
    end
    
    return -1  # Return -1 if the target is not found
end

binary_search (generic function with 1 method)

In [22]:
# L_CC_mass_xf = xf["Final!O:O"][2:end]
# L_MLO_mass_xf = xf["Final!S:S"][2:end]
# R_CC_mass_xf = xf["Final!F:F"][2:end]
# R_MLO_mass_xf = xf["Final!J:J"][2:end]

In [23]:
# Report missing data
valid_data_idx = []
for i = 1 : size(output_csv)[1]
    curr_SID = output_csv[i][1]
    # Check for missing img
    L_CC_notfound = size(output_csv[i][6])[1] == 0
    R_CC_notfound = size(output_csv[i][14])[1] == 0
    L_MLO_notfound = size(output_csv[i][2])[1] == 0
    R_MLO_notfound = size(output_csv[i][10])[1] == 0
    if L_CC_notfound || R_CC_notfound || L_MLO_notfound || R_MLO_notfound
        printstyled(curr_SID; color = :yellow)
        print(": ")
        if L_CC_notfound
            print("[L_CC] ")
        end
        if R_CC_notfound
            print("[R_CC] ")
        end
        if L_MLO_notfound
            print("[L_MLO] ")
        end
        if R_MLO_notfound
            print("[R_MLO] ")
        end
        println("not found.")
        continue
    end
    # Check for missing roi
    idx = binary_search(curr_SID)
    L_CC_notfound = size(output_csv[i][7])[1] == 0
    if L_CC_notfound && L_CC_mass_xf[idx]==0
        push!(output_csv[i][7], "zero BAC")
        L_CC_notfound = false
    end
    R_CC_notfound = size(output_csv[i][15])[1] == 0
    if R_CC_notfound && R_CC_mass_xf[idx]==0
        push!(output_csv[i][15], "zero BAC")
        R_CC_notfound = false
    end
    L_MLO_notfound = size(output_csv[i][3])[1] == 0
    if L_MLO_notfound && L_MLO_mass_xf[idx]==0
        push!(output_csv[i][3], "zero BAC")
        L_MLO_notfound = false
    end
    R_MLO_notfound = size(output_csv[i][11])[1] == 0
    if R_MLO_notfound && R_MLO_mass_xf[idx]==0
        push!(output_csv[i][11], "zero BAC")
        R_MLO_notfound = false
    end

    if L_CC_notfound || R_CC_notfound || L_MLO_notfound || R_MLO_notfound
        printstyled(curr_SID; color = :yellow)
        print(": ")
        if L_CC_notfound
            print("-L_CC- ")
        end
        if R_CC_notfound
            print("-R_CC- ")
        end
        if L_MLO_notfound
            print("-L_MLO- ")
        end
        if R_MLO_notfound
            print("-R_MLO- ")
        end
        println("not found.")
        continue
    end
    # Passed check
    push!(valid_data_idx, i)
end
valid_data_idx = Int.(valid_data_idx)
output_csv = output_csv[valid_data_idx];

SID-100596: -R_CC- not found.
SID-101248: -L_CC- not found.
SID-102408: -L_CC- -R_CC- -L_MLO- -R_MLO- not found.
SID-102418: [L_CC] [R_CC] [L_MLO] [R_MLO] not found.
SID-102735: -R_CC- not found.
SID-104759: -L_CC- -R_CC- -L_MLO- -R_MLO- not found.
SID-105765: [L_CC] [R_CC] [L_MLO] [R_MLO] not found.
SID-106417: [L_CC] [R_CC] [L_MLO] [R_MLO] not found.
SID-106506: [L_CC] [R_CC] [L_MLO] [R_MLO] not found.
SID-108703: [L_CC] [R_CC] [L_MLO] [R_MLO] not found.
SID-109897: [L_CC] [R_CC] [L_MLO] [R_MLO] not found.
SID-110094: [R_MLO] not found.
SID-111639: [L_CC] [R_CC] [L_MLO] [R_MLO] not found.
SID-111873: -R_MLO- not found.
SID-112095: -L_CC- -R_CC- -L_MLO- not found.
SID-112997: [L_CC] [R_CC] [L_MLO] [R_MLO] not found.
SID-118029: [R_CC] not found.
SID-119494: [L_CC] [R_CC] [L_MLO] [R_MLO] not found.
SID-119663: -L_CC- -L_MLO- not found.
SID-119741: [L_CC] [R_CC] [L_MLO] [R_MLO] not found.
SID-120269: [R_CC] not found.
SID-120519: [L_CC] [R_CC] [L_MLO] [R_MLO] not found.
SID-120989: -L_C

In [24]:
# Check for duplicate images
for i = 1 : size(output_csv)[1]
    L_CC_case = deduplicate_images("L CC", i)
    L_MLO_case = deduplicate_images("L MLO", i)
    R_CC_case = deduplicate_images("R CC", i)
    R_MLO_case = deduplicate_images("R MLO", i)

    if L_CC_case == "Error" || L_MLO_case == "Error" || R_CC_case == "Error" || R_MLO_case == "Error"
        print(i, " ")
        printstyled("Error!"; color = :yellow)
        println()
        break
    end
    if L_CC_case == "Passed" && L_MLO_case == "Passed" && R_CC_case == "Passed" && R_MLO_case == "Passed"
        continue
    end
    
    print("$i ")
    printstyled("Passed"; color = :green)
    println(": $L_CC_case; $L_MLO_case; $R_CC_case; $R_MLO_case")
end

12 Passed: Passed; Case 1.3; Passed; Passed
14 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
17 Passed: Passed; Case 2; Passed; Case 2
23 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
27 Passed: Case 1.3; Case 1.3; Case 1.3; Case 1.3
31 Passed: Case 2; Case 2; Passed; Passed
34 Passed: Passed; Passed; Passed; Case 1.1
40 Passed: Passed; Passed; Case 1.1; Passed
49 Passed: Passed; Passed; Case 1.1; Passed
53 Passed: Passed; Passed; Passed; Case 1.3
59 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
62 Passed: Passed; Case 1.1; Passed; Passed
64 Passed: Passed; Passed; Passed; Case 1.1
69 Passed: Case 1.1; Passed; Case 1.1; Passed
70 Passed: Passed; Case 1.1; Passed; Case 1.3
71 Passed: Passed; Case 1.1; Passed; Passed
74 Passed: Passed; Passed; Passed; Case 1.1
78 Passed: Case 2; Case 2; Case 2; Case 2
88 Passed: Case 1.3; Passed; Passed; Passed
89 Passed: Passed; Case 1.1; Passed; Passed
91 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
96 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.

665 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
669 Passed: Passed; Passed; Passed; Case 1.1
670 Passed: Case 1.3; Case 1.3; Case 1.3; Case 1.3
673 Passed: Passed; Passed; Case 1.1; Passed
677 Passed: Passed; Case 1.1; Passed; Passed
680 Passed: Passed; Case 1.1; Passed; Case 1.1
682 Passed: Passed; Passed; Case 1.1; Passed
685 Passed: Passed; Passed; Passed; Case 1.3
687 Passed: Case 1.3; Case 1.3; Case 1.3; Case 1.3
688 Passed: Case 1.1; Passed; Passed; Passed
691 Passed: Passed; Case 1.3; Passed; Passed
701 Passed: Passed; Case 1.3; Passed; Case 1.3
709 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
718 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
722 Passed: Passed; Case 1.1; Passed; Case 1.1
724 Passed: Passed; Case 1.1; Passed; Case 1.3
727 Passed: Passed; Case 1.1; Passed; Case 1.1
728 Passed: Passed; Case 1.1; Passed; Case 1.1
732 Passed: Passed; Case 1.1; Passed; Case 1.1
733 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
739 Passed: Passed; Passed; Passed; Case 1.1
741 P

1719 Passed: Passed; Case 1.1; Passed; Passed
1728 Passed: Passed; Passed; Passed; Case 1.1
1732 Passed: Passed; Case 3.2; Passed; Passed
1742 Passed: Passed; Case 1.1; Passed; Passed
1744 Passed: Passed; Passed; Case 1.1; Passed
1768 Passed: Passed; Case 1.1; Passed; Case 1.1
1770 Passed: Case 2; Case 2; Passed; Passed
1772 Passed: Passed; Passed; Case 1.3; Passed
1773 Passed: Passed; Case 1.3; Passed; Passed
1779 Passed: Passed; Passed; Passed; Case 3.2
1780 Passed: Case 1.1; Case 1.1; Case 1.3; Case 1.3
1793 Passed: Case 2; Case 2; Case 2; Case 2
1803 Passed: Passed; Passed; Case 1.1; Passed
1810 Passed: Passed; Passed; Case 2; Case 2
1811 Passed: Passed; Passed; Case 1.1; Passed
1818 Passed: Passed; Case 1.3; Passed; Passed
1820 Passed: Passed; Case 2; Passed; Case 1.1
1821 Passed: Passed; Passed; Passed; Case 1.1
1823 Passed: Passed; Passed; Passed; Case 1.1
1829 Passed: Passed; Passed; Case 1.3; Passed
1833 Passed: Case 1.3; Passed; Passed; Passed
1840 Passed: Passed; Case 1.1; P

2524 Passed: Passed; Passed; Passed; Case 1.1
2526 Passed: Passed; Passed; Case 1.1; Passed
2528 Passed: Passed; Passed; Passed; Case 1.1
2535 Passed: Passed; Case 1.1; Case 1.1; Case 1.1
2548 Passed: Passed; Case 1.1; Passed; Passed
2549 Passed: Case 1.1; Passed; Passed; Passed
2550 Passed: Case 1.1; Passed; Passed; Passed
2551 Passed: Passed; Passed; Passed; Case 1.1
2552 Passed: Passed; Case 1.1; Passed; Passed
2563 Passed: Passed; Case 1.1; Case 1.1; Passed
2564 Passed: Passed; Case 1.1; Passed; Passed
2573 Passed: Passed; Passed; Passed; Case 1.1
2575 Passed: Passed; Passed; Passed; Case 1.1
2580 Passed: Passed; Passed; Passed; Case 1.1
2582 Passed: Case 1.1; Passed; Case 1.1; Passed
2584 Passed: Passed; Case 1.1; Passed; Case 1.1
2586 Passed: Passed; Case 1.1; Passed; Case 1.1
2588 Passed: Passed; Case 1.1; Passed; Passed
2589 Passed: Passed; Case 1.1; Passed; Case 1.1
2592 Passed: Passed; Passed; Passed; Case 1.1
2597 Passed: Passed; Case 1.1; Passed; Case 1.1
2598 Passed: Case 

: Passed; Passed; Passed; Case 1.3
3216 Passed: Case 1.1; Passed; Passed; Passed
3225 Passed: Case 2; Case 2; Case 2; Case 2
3240 Passed: Passed; Passed; Case 1.3; Passed
3244 Passed: Passed; Case 1.1; Case 1.1; Case 1.1
3246 Passed: Passed; Case 1.1; Passed; Case 1.1
3248 Passed: Case 1.3; Passed; Case 1.3; Passed
3250 Passed: Passed; Passed; Case 1.3; Passed
3251 Passed: Passed; Passed; Passed; Case 1.3
3254 Passed: Case 1.1; Passed; Case 1.1; Passed
3260 Passed: Case 1.1; Passed; Case 1.1; Passed
3262 Passed: Case 1.1; Passed; Passed; Passed
3269 Passed: Passed; Case 1.1; Passed; Case 1.1
3272 Passed: Passed; Passed; Passed; Case 1.3
3274 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
3279 Passed: Passed; Case 1.1; Passed; Case 1.1
3281 Passed: Passed; Case 1.1; Passed; Passed
3284 Passed: Passed; Passed; Passed; Case 1.1
3287 Passed: Passed; Passed; Case 1.1; Passed
3288 Passed: Passed; Case 1.1; Passed; Passed
3293 Passed: Passed; Passed; Case 1.1; Passed
3298 Passed: Case 1.1; Pa

3843 Passed: Passed; Case 1.1; Passed; Passed
3848 Passed: Passed; Case 1.1; Passed; Passed
3850 Passed: Passed; Passed; Passed; Case 1.1
3855 Passed: Passed; Passed; Passed; Case 1.1
3859 Passed: Passed; Case 1.1; Passed; Case 1.1
3860 Passed: Case 2; Case 2; Case 2; Case 2
3861 Passed: Passed; Passed; Case 1.3; Passed
3869 Passed: Passed; Case 1.3; Passed; Passed
3872 Passed: Passed; Case 1.1; Passed; Passed
3876 Passed: Passed; Passed; Case 1.1; Passed
3886 Passed: Case 2; Case 2; Case 2; Case 2
3887 Passed: Passed; Passed; Case 1.1; Passed
3888 Passed: Passed; Case 1.1; Passed; Passed
3893 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
3894 Passed: Passed; Case 1.1; Passed; Case 1.1
3897 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
3904 Passed: Case 1.1; Passed; Passed; Passed
3906 Passed: Passed; Case 1.1; Case 1.1; Case 1.1
3908 Passed: Case 1.1; Case 1.1; Case 1.1; Passed
3910 Passed: Passed; Passed; Passed; Case 1.1
3911 Passed: Case 1.1; Passed; Passed; Passed
3914 Passed: P

4116 Passed: Case 3.1; Case 3.1; Case 3.1; Case 3.1
4123 Passed: Passed; Passed; Case 1.1; Passed
4127 Passed: Passed; Case 1.1; Passed; Passed
4128 Passed: Case 1.3; Case 1.3; Case 1.1; Passed
4134 Passed: Passed; Case 1.1; Passed; Case 1.1
4138 Passed: Case 1.1; Passed; Passed; Passed
4139 Passed: Passed; Passed; Passed; Case 1.1
4146 Passed: Passed; Case 1.1; Passed; Case 1.1
4147 Passed: Passed; Passed; Passed; Case 1.1
4148 Passed: Case 1.1; Passed; Passed; Passed
4150 Passed: Case 1.3; Case 1.3; Case 1.3; Case 1.1
4154 Passed: Passed; Passed; Passed; Case 1.1
4160 Passed: Passed; Case 1.1; Case 1.1; Case 1.1
4164 Passed: Passed; Passed; Case 1.1; Passed
4169 Passed: Passed; Passed; Passed; Case 1.1
4170 Passed: Case 2; Case 2; Case 2; Case 2
4175 Passed: Passed; Case 1.1; Passed; Passed
4176 Passed: Passed; Case 1.1; Passed; Passed
4179 Passed: Passed; Case 1.3; Passed; Case 1.1
4180 Passed: Passed; Passed; Case 1.3; Passed
4183 Passed: Passed; Case 1.1; Passed; Passed
4188 Passe

4804 Passed: Case 2; Case 2; Case 2; Case 2
4805 Passed: Case 1.1; Passed; Case 1.3; Passed
4806 Passed: Passed; Case 1.1; Passed; Case 1.3
4808 Passed: Passed; Passed; Case 1.1; Case 1.1
4809 Passed: Case 2; Case 2; Case 2; Case 2
4811 Passed: Case 3.1; Case 2; Case 3.1; Case 2
4812 Passed: Case 1.1; Case 1.3; Case 1.1; Case 1.3
4813 Passed: Case 1.1; Passed; Case 1.1; Passed
4815 Passed: Case 2; Passed; Passed; Case 2
4816 Passed: Case 2; Case 2; Case 2; Case 2
4817 Passed: Passed; Case 1.1; Passed; Passed
4818 Passed: Case 3.1; Case 1.1; Case 1.1; Case 1.1
4825 Passed: Case 1.1; Passed; Passed; Passed
4826 Passed: Passed; Case 2; Passed; Passed
4828 Passed: Case 1.1; Passed; Case 1.3; Passed
4829 Passed: Passed; Case 1.1; Passed; Passed
4830 Passed: Case 1.1; Case 1.3; Case 1.1; Case 1.1
4831 Passed: Case 2; Case 3.2; Passed; Passed
4832 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
4834 Passed: Passed; Case 2; Passed; Passed
4837 Passed: Passed; Case 1.1; Passed; Passed
4839 Passe

5098 Passed: Case 1.1; Case 1.1; Case 1.1; Case 1.1
5101 Passed: Passed; Passed; Passed; Case 1.1
5106 Passed: Passed; Case 1.3; Passed; Case 1.3
5108 Passed: Passed; Passed; Passed; Case 1.1


In [28]:
@save "result_from_step1.jld" output_csv